In [1]:
import torch

from models import S2SPL
from dataset import en_de_dataset
from utils import load_config_json
import os
from torchtext.data.metrics import bleu_score

C:\Users\83577\anaconda3\envs\torch\lib\site-packages\torchtext\data\utils.py:105: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(
C:\Users\83577\anaconda3\envs\torch\lib\site-packages\torchtext\data\utils.py:105: UserWarning: Spacy model "de" could not be loaded, trying "de_core_news_sm" instead
  warnings.warn(


In [2]:
path = "DEVICEcuda BATCH_SIZE128 LEARNING_RATE0.0001 EPOCHS500 ENCODER_TYPEGRU DECODER_TYPEGRU ATTENTION_HEAD1 ENCODER_LAYERS2 DECODER_LAYERS2 EMB_DIM150 HIDDEN_ENCODER512 HIDDEN_DECODER512 DROPOUT_ENCODER0.5 DROPOUT_DECODER0.5 BIDIRECTIONFalse PAIR('de', 'en')"  #os.listdir('best')[-1] # could be replaced by the folder name
path = os.path.join('best', path)

checkpoint = '-epoch=121-valid_loss=2.95-validation.ckpt'#os.listdir(path)[0] # could be replaced by your own
checkpoint = os.path.join(path, checkpoint)

params = os.path.join(path, 'config.json')

print(path + '\n')
print(checkpoint + '\n')
print(params)

# Initialize args
args = load_config_json(params)

args['DEVICE'] = 'cpu'

best\DEVICEcuda BATCH_SIZE128 LEARNING_RATE0.0001 EPOCHS500 ENCODER_TYPEGRU DECODER_TYPEGRU ATTENTION_HEAD1 ENCODER_LAYERS2 DECODER_LAYERS2 EMB_DIM150 HIDDEN_ENCODER512 HIDDEN_DECODER512 DROPOUT_ENCODER0.5 DROPOUT_DECODER0.5 BIDIRECTIONFalse PAIR('de', 'en')

best\DEVICEcuda BATCH_SIZE128 LEARNING_RATE0.0001 EPOCHS500 ENCODER_TYPEGRU DECODER_TYPEGRU ATTENTION_HEAD1 ENCODER_LAYERS2 DECODER_LAYERS2 EMB_DIM150 HIDDEN_ENCODER512 HIDDEN_DECODER512 DROPOUT_ENCODER0.5 DROPOUT_DECODER0.5 BIDIRECTIONFalse PAIR('de', 'en')\-epoch=121-valid_loss=2.95-validation.ckpt

best\DEVICEcuda BATCH_SIZE128 LEARNING_RATE0.0001 EPOCHS500 ENCODER_TYPEGRU DECODER_TYPEGRU ATTENTION_HEAD1 ENCODER_LAYERS2 DECODER_LAYERS2 EMB_DIM150 HIDDEN_ENCODER512 HIDDEN_DECODER512 DROPOUT_ENCODER0.5 DROPOUT_DECODER0.5 BIDIRECTIONFalse PAIR('de', 'en')\config.json


In [3]:
# Initial Dataset
trainset = en_de_dataset(split='train', pair=args['PAIR'])
# Initialize model parameters
s2s_model = S2SPL(
        vocab_size_encoder=len(trainset.input_vocab),
        vocab_size_decoder=len(trainset.target_vocab),
        input_vocab=trainset.input_vocab,
        target_vocab=trainset.target_vocab,
        input_id_to_word=trainset.id_to_word_input,
        target_id_to_word=trainset.id_to_word_target,
        args=args
)


model = s2s_model.load_from_checkpoint(checkpoint)
model.eval()

Input vocab size: 8014
Target vocab size: 6191


S2SPL(
  (encoder): RNNEncoder(
    (emb): Embedding(8014, 150)
    (rnn): GRU(150, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): RNNDecoder(
    (emb): Embedding(6191, 150)
    (rnn): GRU(150, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
    (decode): Linear(in_features=512, out_features=6191, bias=True)
  )
  (cross_entrophy): CrossEntropyLoss()
)

In [29]:
output = model.translate('Ein Mann geht am Strasse', 'A man walking on the street.' )

output

('A man walking by the railroad .', 0.0)

In [32]:
input_pair, target_pair = trainset.random_pairs()
print((input_pair, target_pair))
print(model.translate(input_pair, target_pair))

('Eine Frau, die auf einem Platz einen Tennisschläger hält.', 'A woman holding a tennis racket on a court.')
('A woman swinging a tennis racket on a court .', 0.45180100202560425)


In [292]:
import math
def attn(query, key, value, input_indice):

        input_indice_ext = input_indice.unsqueeze(1).repeat(1, query.shape[0], 1)
        query = query.permute(1, 0, 2) # [seq_dec, batch, features] -> [batch, seq_dec, features]
        key = key.permute(1, 2, 0) # [seq_enc, batch, features] -> [batch, features, seq_enc]
        value =  value.permute(1, 0, 2)
        # Getting attention score with mask
        score = torch.bmm(query, key)/ math.sqrt(query.shape[1]) # [batch, seq_dec, seq_enc]
        score[input_indice_ext == 0] = -9e3


        score = torch.softmax(score, dim=2)
        print(score)

        attn_output = torch.bmm(score, value)
        print(attn_output.shape)
        return attn_output.permute(1, 0, 2)



query = torch.randn(2, 2, 300)

input_indice = torch.randint(4, 9000, (2, 10))
input_indice[0, -4:] = 0
input_indice[1, -6:] = 0
input_indice[0, -4] = 3
input_indice[1, -6] = 3
key = torch.randn(10, 2, 300)
value = key

print(attn(query, key, value, input_indice).shape)
print(key[(input_indice == 3).T].unsqueeze(0).shape, key[-1].unsqueeze(0).shape)

tensor([[[4.1490e-21, 2.8758e-12, 6.0386e-10, 2.8572e-04, 4.3376e-07,
          5.2479e-02, 9.4723e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [4.1188e-14, 1.7602e-19, 7.9917e-13, 1.0915e-04, 4.2903e-12,
          4.4508e-20, 9.9989e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00]],

        [[1.2221e-06, 9.9147e-01, 4.7685e-04, 8.0489e-03, 1.2331e-10,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [4.0651e-15, 1.0000e+00, 9.2983e-19, 6.2053e-09, 6.2126e-21,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00]]])
torch.Size([2, 2, 300])
torch.Size([2, 2, 300])
torch.Size([1, 2, 300]) torch.Size([1, 2, 300])
